# CPU Emulator

## Simple CPU Emulator

In [ ]:
class SimpleCPU:
    
    def __init__(self):
        self.registers = [0]*4  # R0, R1, R2, R3
        self.PC = 0 
        self.program_memory = [
            ("MOV", 0, 10), 
            ("MOV", 1, 20),
            ("ADD", 2, 0),
            ("ADD", 2, 1),
            ("SUB", 2, 0),
            ("HALT", 0,0)
        ]
        
    def fetch(self):
        
        if self.PC >= len(self.program_memory):
            return None
        
        instruction = self.program_memory[self.PC]
        self.PC += 1
        
        return instruction
    
    
    def decode(self, instruction):
        opcode, op1, op2 = instruction
        
        if opcode == "MOV":
            self.registers[op1] = op2
            print(f"Execution : R{op1} = {op2}")
            
        elif opcode == "ADD":
            self.registers[op1] += self.registers[op2]
            print(f"Execution: R{op1} += R{op2} -> {self.registers[op1]}")
            
        elif opcode == "SUB":
            self.registers[op1] -= self.registers[op2]
            print(f"Execution: R{op1} -= R{op2} -> {self.registers[op1]}")
            
        elif opcode == "HALT":
            return False
        
        return True
    
    
    def run(self):
        running = True
        cycle = 0
        
        while running:
            print(f"--- Cycle {cycle} | PC: {self.PC} ---")
            instruction = self.fetch()
            
            if instruction:
                running = self.decode(instruction)
            
            else:
                running = False
            
            cycle +=1

In [2]:
cpu = SimpleCPU()

cpu.run()

--- Cycle 0 | PC: 0 ---
Execution : R0 = 10
--- Cycle 1 | PC: 1 ---
Execution : R1 = 20
--- Cycle 2 | PC: 2 ---
Execution: R2 += R0 -> 10
--- Cycle 3 | PC: 3 ---
Execution: R2 += R1 -> 30
--- Cycle 4 | PC: 4 ---
Execution: R2 -= R0 -> 20
--- Cycle 5 | PC: 5 ---


## Complex CPU Emulator

- Let's make a CPU emulator able to print Fibonacci number up to a given number `n`.

- The Fibonacci sequence is defined as follows:

$$ F(0) = 0, F(1) = 1, F(n) = F(n-1) + F(n-2) \text{ for } n > 1 $$

- The CPU emulator will have a simple instruction set that includes basic arithmetic operations, memory access, and control flow instructions.

- We will define a simple assembly-like language for our CPU emulator, and then implement the emulator in Python.
  
- The instruction set will include the following instructions:
  - `MOV R, X`: Move the value `X` into register `R`.
  - `LOAD R, X`: Load the value at memory address `X` into register `R`.
  - `ADD R1, R2, R3`: Add the values in registers `R1` and `R2`, and store the result in register `R3`.
  - `SUB R1, R2, R3`: Subtract the value in register `R2` from the value in register `R1`, and store the result in register `R3`.
  - `JMP X`: Jump to the instruction at memory address `X`.   
  - `JZ R, X`: Jump to the instruction at memory address `X` if the value in register `R` is zero.
  - `PRINT R`: Print the value in register `R`.
  - `HALT`: Stop the execution of the program.


### CPU

In [27]:
# lets make a CPU that accepts user input for instructions and performs the same operations as above.

class CPU:
    
    def __init__(self):
        self.registers = [0]*5  # R0, R1, R2, R3, R4
        self.PC = 0  # program counter set to sero initially
        self.program_memory = []
        
        
    def fetch(self):
        if self.PC >= len(self.program_memory):
            return False
        
        instruction = self.program_memory[self.PC]
        self.PC += 1
        
        return instruction  
    
    def decode(self, instruction, execution_log=False):
        '''
         `MOV R, X`: Move the value `X` into register `R`.
         `LOAD R, X`: Load the value at memory address `X` into register `R`.
        `ADD R1, R2, R3`: Add the values in registers `R1` and `R2`, and store the result in register `R3`.
        `SUB R1, R2, R3`: Subtract the value in register `R2` from the value in register `R1`, and store the result in register `R3`.
        `JMP X`: Jump to the instruction at memory address `X`.   
        `JZ R, X`: Jump to the instruction at memory address `X` if the value in register `R` is zero.
        `PRINT R`: Print the value in register `R`.
        `HALT`: Stop the execution of the program.
        '''
        opcode, op1, op2 = instruction
        
        if opcode == "MOV":
            self.registers[op1] = op2
            if execution_log:
                print(f"Execution : R{op1} = {op2}")
                
        elif opcode == "ADD":
            self.registers[op1] += self.registers[op2]
            if execution_log:
                print(f"Execution: R{op1} += R{op2} -> {self.registers[op1]}")
                
        elif opcode == "SUB":
            self.registers[op1] -= self.registers[op2]
            if execution_log:
                print(f"Execution: R{op1} -= R{op2} -> {self.registers[op1]}")
        
        elif opcode == "JMP":
            self.PC = op1
        
        elif opcode == "JZ":
            if self.registers[op1] == 0:
                self.PC = op2
        
        elif opcode == "PRINT":
            print(f"{self.registers[op1]}")
                
        elif opcode == "HALT":
            return False

        return True
    
    def run(self, execution_log=False):
        running = True
        cycle = 0 
        
        while running:
            if execution_log:
                print(f"---Cycle {cycle} | PC: {self.PC} ---")
            instruction = self.fetch()
            
            if instruction:
                running = self.decode(instruction, execution_log)
                
            else: 
                running = False
                
            cycle += 1
        

### Program to compute Fibonacci numbers up to `n`:

In [38]:
cpu = CPU()
n = int(input("Enter the value of n to compute Fibonacci numbers up to n: "))

# lets write a program to compute Fibonacci numbers up to `n`:

cpu.program_memory = [
    ("MOV",0,n-1),           # counter variable
    ("MOV", 1, 0),           # initial condition f[0]
    ("MOV", 2, 1),           # initial condition f[1]
    ("MOV", 3, 1),           # to reduce the counter lets keep this
    ("PRINT",2,None),        # print f
    ("JZ", 0, 13),           # if counter is zero jump to end
    ("SUB",0,3),             #reducing counter
    ("ADD",4,2 ),            # store f[i-2] in R4
    ("ADD",2,1),             # compute f[i] = f[i-1] + f[i-2]
    ("SUB",1,1),            
    ("ADD",1,4),             # move f[i-2] to f[i-1]
    ("SUB",4,4),
    ("JMP",4, None),         # jump back to print f and check counter
    ("HALT",0,0)             # end of program
]

cpu.run()

1
1
2
3
5
8
13
21
